In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [6]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [7]:
DATADIR = 'UCI_HAR_Dataset'
# Raw data signals
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [8]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)


def load_signals(subset):
    signals_data = []
    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(_read_csv(filename).values)

    return np.transpose(signals_data, (1, 2, 0))


def load_y(subset):
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).values

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')
    return X_train, X_test, y_train, y_test


In [9]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])


def _count_classes(y):
    return y.shape[1]

n_classes = _count_classes(Y_train)

/var/folders/0p/wjrkk41s21j0vmmlh60szvpr0000gn/T/ipykernel_59102/1890298597.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
/var/folders/0p/wjrkk41s21j0vmmlh60szvpr0000gn/T/ipykernel_59102/1890298597.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
/var/folders/0p/wjrkk41s21j0vmmlh60szvpr0000gn/T/ipykernel_59102/1890298597.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
/var/folders/0p/wjrkk41s21j0vmmlh60szvpr0000gn/T/ipykernel_59102/1890298597.py:2: FutureWarning: The 'delim_whitespace' keyword

In [10]:
# Initialize parameters
epochs = 30
batch_size = 16
n_hidden = 32  # Define n_hidden here

In [11]:
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))  # Corrected activation function
model.summary()


/Users/sohanm/EMEA/PET/HAR/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,574 (21.77 KB)

 Trainable params: 5,574 (21.77 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [13]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,
          verbose=1)

Epoch 1/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.4595 - loss: 1.3075 - val_accuracy: 0.6546 - val_loss: 0.8824
Epoch 2/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6367 - loss: 0.8068 - val_accuracy: 0.6793 - val_loss: 0.7630
Epoch 3/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6840 - loss: 0.7023 - val_accuracy: 0.7533 - val_loss: 0.6866
Epoch 4/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7492 - loss: 0.5923 - val_accuracy: 0.7740 - val_loss: 0.6334
Epoch 5/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7808 - loss: 0.5147 - val_accuracy: 0.7964 - val_loss: 0.5735
Epoch 6/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8171 - loss: 0.4509 - val_accuracy: 0.8090 - val_loss: 0.5672
Epoch 7/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8715 - loss: 0.3926 - val_accuracy: 0.8025 - val_loss: 0.5624
Epoch 8/30
460/460 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8899 - loss: 0.3401 - val_accu

In [14]:
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      384       106        0                   1   
STANDING                 0       95       437        0                   0   
WALKING                  0        0         1      475                   8   
WALKING_DOWNSTAIRS       0        0         1        8                 409   
WALKING_UPSTAIRS         1        0         0        5                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           12  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 465  


In [15]:
score = model.evaluate(X_test, Y_test)
print(f'Accuracy: {score[1]*100:.2f}%, Loss: {score[0]:.2f}')

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8753 - loss: 0.3659
Accuracy: 91.86%, Loss: 0.30
